In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

/home/duc/anaconda3/envs/interpreter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL_ID  = "deepseek-ai/deepseek-coder-33b-instruct"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(MODEL_ID, trust_remote_code=True, torch_dtype=torch.float16, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
messages = [
    { 'role': 'user', 'content': "write a quick sort algorithm in python."}
]

In [6]:
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")

In [7]:
tokenizer.decode(inputs[0])

'<｜begin▁of▁sentence｜>You are an AI programming assistant, utilizing the Deepseek Coder model, developed by Deepseek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer\n### Instruction:\nwrite a quick sort algorithm in python.\n'

In [8]:
torch.cuda.is_available()

True

In [9]:
torch.cuda.device_count()

2

In [10]:
pipeline_deepseek = pipeline(
        "text-generation",
        model=MODEL_ID,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        model_kwargs={
          "load_in_4bit":True,
            # "torch_dtype":torch.float16,
           "bnb_4bit_compute_dtype":torch.float16,
            "device_map":"auto",
            }
    )

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 7/7 [00:25<00:00,  3.65s/it]


In [11]:
response = pipeline_deepseek(tokenizer.decode(inputs[0]))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


In [12]:
response

[{'generated_text': '<｜begin▁of▁sentence｜>You are an AI programming assistant, utilizing the Deepseek Coder model, developed by Deepseek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer\n### Instruction:\nwrite a quick sort algorithm in python.\n\nIn your explanation, try to keep the concept of divide and conquer strategy.\n\nAlso, describe the time complexity of quick sort in big O notation.\n'}]

In [13]:
print(response[0]["generated_text"])

<｜begin▁of▁sentence｜>You are an AI programming assistant, utilizing the Deepseek Coder model, developed by Deepseek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer
### Instruction:
write a quick sort algorithm in python.

In your explanation, try to keep the concept of divide and conquer strategy.

Also, describe the time complexity of quick sort in big O notation.

